<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/ML_in_Business_HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1

https://archive.ics.uci.edu/dataset/5/arrhythmia

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("arrhythmia.data", header=None)
df.head(3)

In [ ]:
df.iloc[:,279].nunique(), np.sort(df.iloc[:,279].unique())

Согласно описанию существует 16 групп классификации наличия или отстутсвия сердечной аритмии.

01 - Normal 245

02 - Ischemic changes (Coronary Artery Disease) 44

03 - Old Anterior Myocardial Infarction 15

04 - Old Inferior Myocardial Infarction 15

05 - Sinus tachycardy 13

06 - Sinus bradycardy 25

07 - Ventricular Premature Contraction (PVC) 3

08 - Supraventricular Premature Contraction 2

09 - Left bundle branch block 9

10 - Right bundle branch block 50

11 - 1. degree AtrioVentricular block 0

12 - 2. degree AV block 0

13 - 3. degree AV block 0

14 - Left ventricule hypertrophy 4

15 - Atrial Fibrillation or Flutter 5

16 - Others 22

Будем работать со второй группой 02 - Ischemic changes (Coronary Artery Disease)

In [ ]:
data = df.copy()
data[279].replace([1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 0, inplace=True)
data[279].replace(2, 1, inplace=True)
data[279]

In [ ]:
print(data.shape)

In [ ]:
data.iloc[:, -1].value_counts()

In [ ]:
def get_num_features(df):
    """
    функция возвращает список числовых признаков
    :param df
    :return: list
    """
    numerical_features = df.select_dtypes(include=[np.number])
    # print(f"count of numeric_features_train {numerical_features.shape[1]}")
    return numerical_features.columns


def get_cat_features(df):
    """
    функция возвращает список признаков с типом "объект"
    :param df:
    :return: list
    """
    categorical_features = df.select_dtypes(include=[np.object])
    return categorical_features.columns

In [ ]:
get_num_features(data)

In [ ]:
get_cat_features(data)

2

In [ ]:
data[10].unique()

In [ ]:
for col in [10, 11, 12, 13, 14]:
    # извлекаем индексы по условию (тех элементов, которые имеют значение "?")
    indices = np.where(data[col] != '?')
    # получаем спиcок элементов по индексам [0], приводим к строке, к целому и находим медиану
    col_med = np.median(np.take(data[col], indices[0]).astype(str).astype(int))
    # заменяем некорректное значение медианой
    data[col] = data[col].replace('?', col_med)

In [ ]:
data[10].unique()

In [ ]:
for col in [10, 11, 12, 13, 14]:
    data[col] = data[col].astype(int)

3

In [ ]:
from sklearn.model_selection import train_test_split

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [ ]:
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

Проверяем качество модели

In [ ]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0))
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0))
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0))
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0))
    return f1, roc, rec, prc

f1, roc, rec, prc = evaluate_results(y_test, y_predict)

score = []
score.append([f1, prc, rec, roc])

4

In [ ]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

In [ ]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

In [ ]:
mod_data[279].values

In [ ]:
mod_data.head(10)

In [ ]:
x_data = mod_data.iloc[:,:-2].values # just the X
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

5

In [ ]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

In [ ]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values,
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

f1, prc, rec, roc = evaluate_results(sample_test.iloc[:,-2].values, y_predict)

6

In [ ]:
score.append([f1, prc, rec, roc])

In [ ]:
df = pd.DataFrame({'original': score[0], 'random negative sampling': score[1]}).T
df.columns = ['fscore', 'precision', 'recall', 'roc_auc']
df

7

In [ ]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 30% of the positives marked
pos_sample_len = int(np.ceil(0.3 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

In [ ]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

In [ ]:
mod_data[279].values

In [ ]:
mod_data.head(10)

In [ ]:
x_data = mod_data.iloc[:,:-2].values # just the X
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [ ]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

In [ ]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values,
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

f1, prc, rec, roc = evaluate_results(sample_test.iloc[:,-2].values, y_predict)

In [ ]:
score.append([f1, prc, rec, roc])

In [ ]:
df = pd.DataFrame({'original': score[0], 'random negative sampling': score[1], 'random negative sampling 0.3': score[2]}).T
df.columns = ['fscore', 'precision', 'recall', 'roc_auc']
df

In [ ]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 20% of the positives marked
pos_sample_len = int(np.ceil(0.2 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

In [ ]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

In [ ]:
x_data = mod_data.iloc[:,:-2].values # just the X
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [ ]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

In [ ]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values,
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

f1, prc, rec, roc = evaluate_results(sample_test.iloc[:,-2].values, y_predict)

In [ ]:
score.append([f1, prc, rec, roc])

In [ ]:
df = pd.DataFrame({'original': score[0],
                   'random negative sampling': score[1],
                   'random negative sampling 0.3': score[2],
                   'random negative sampling 0.2': score[3]}).T
df.columns = ['fscore', 'precision', 'recall', 'roc_auc']
df

Вывод: наилучший F-score у обычного XGboost, однако Recall и Precision выше при использовании random negative sampling с долей 0.2